In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
from keras.models import Model, load_model
from keras.layers import Dense, Embedding, Input, GRU
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout,GlobalAveragePooling1D,Conv1D
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint
from models_def import Attention

max_features = 180000
maxlen = 150

def clean_text( text ):
    text = text.lower().split()
    text = " ".join(text)
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+\-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    #
    return text

train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

list_sentences_train = train["comment_text"].fillna("CVxTz").apply(clean_text).values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("CVxTz").apply(clean_text).values
print(y.shape)


tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_train = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
X_test = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)

print(X_train.shape,X_test.shape)

# check word_index
tmp_cnt = 0
for k in tokenizer.word_index:
    print(k,tokenizer.word_index[k])
    tmp_cnt += 1
    if tmp_cnt >5:
        break
word_idx = tokenizer.word_index

# read word2vec
# 
word_vec_dict = {}
with open('../glove.840B.300d.txt') as f:
    for line in f:
        v_list = line.strip().split(' ')
        k = str(v_list[0])
        v = np.array([float(x) for x in v_list[1:]])
        word_vec_dict[k] = v
print(len(word_vec_dict))
print('Preparing embedding matrix')
EMBEDDING_DIM = 300
nb_words = min(max_features,len(word_idx))
embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word,i in word_idx.items():
    if i >= max_features:
        continue
    else:
        if word in word_vec_dict:
            embedding_matrix[i] = word_vec_dict[word]
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))
del word_vec_dict

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


(159571, 6)
(159571, 150) (153164, 150)
costs 4195
figurines 61515
outgunned 55052
sai 12096
umbridge 38108
hatkirby 154106
2196007
Preparing embedding matrix
Null word embeddings: 76050


In [2]:
from sklearn.metrics import log_loss,accuracy_score
from keras.models import Model
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, concatenate
from keras.layers import CuDNNLSTM, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.preprocessing import text, sequence
from keras.callbacks import Callback

def eval_val(y,train_x):
    res = 0
    acc_res = 0
    for i in range(6):
        curr_loss = log_loss(y[:,i],train_x[:,i])
        acc = accuracy_score(y[:,i],train_x[:,i].round())
        print(i,curr_loss,acc)
        res += curr_loss
        acc_res += acc
    print('final',res/6, acc_res/6)

def get_model():
    inp = Input(shape=(maxlen, ))
    x = Embedding(nb_words, EMBEDDING_DIM, weights=[embedding_matrix],trainable=False)(inp)
    x = Dropout(0.4)(x)
    x = Bidirectional(CuDNNLSTM(64, return_sequences=True))(x)
    x = Attention(maxlen)(x)
    x = Dropout(0.1)(x)
    x = Dense(256, activation="relu")(x)
    x = Dropout(0.1)(x)
    x = Dense(6, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model
print('def model done')

def model done


In [3]:
from sklearn.model_selection import KFold
def kf_train(fold_cnt=3,rnd=1):
    kf = KFold(n_splits=fold_cnt, shuffle=False, random_state=233*rnd)
    train_pred, test_pred = np.zeros((159571,6)),np.zeros((153164,6))
    for train_index, test_index in kf.split(X_train):
        # x,y
        curr_x,curr_y = X_train[train_index],y[train_index]
        hold_out_x,hold_out_y = X_train[test_index],y[test_index]
        
        # model
        model = get_model()
        batch_size = 64
        epochs = 10
        file_path="weights_base.best.h5"
        checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
        callbacks_list = [checkpoint] 
        
        # train and pred
        model.fit(curr_x, curr_y, 
                  batch_size=batch_size, epochs=epochs, 
                  validation_data=(hold_out_x,hold_out_y), 
                  callbacks=callbacks_list)
        
        model.load_weights(file_path)
        y_test = model.predict(X_test)
        test_pred += y_test
        hold_out_pred = model.predict(hold_out_x)
        train_pred[test_index] = hold_out_pred
    test_pred = test_pred / fold_cnt
    print('-------------------------------')
    print('all eval')
    eval_val(y,train_pred)
    return train_pred, test_pred


print('def done')

def done


In [4]:
import pickle
sample_submission = pd.read_csv("../input/sample_submission.csv")

train_pred,test_pred = kf_train(fold_cnt=5,rnd=4)
print(train_pred.shape,test_pred.shape)    

# 40000,150,lstm + global max_pool
# final 0.0407274256871 0.984048897774
# 100000,150 lstm + attention
# final 0.0404159162853 0.984188856371, pub 9849

# 180000,150, spacial dr 0.4, fold 5
# final 0.040301220341891576 0.9844144612742918

Train on 127656 samples, validate on 31915 samples
Epoch 1/10
127656/127656 [==============================] - 77s 601us/step - loss: 0.0635 - acc: 0.9783 - val_loss: 0.0478 - val_acc: 0.9818
Epoch 2/10
127656/127656 [==============================] - 76s 599us/step - loss: 0.0469 - acc: 0.9823 - val_loss: 0.0437 - val_acc: 0.9829
Epoch 3/10
127656/127656 [==============================] - 77s 600us/step - loss: 0.0442 - acc: 0.9831 - val_loss: 0.0426 - val_acc: 0.9833
Epoch 4/10
127656/127656 [==============================] - 77s 601us/step - loss: 0.0419 - acc: 0.9837 - val_loss: 0.0419 - val_acc: 0.9834
Epoch 5/10
127656/127656 [==============================] - 77s 602us/step - loss: 0.0405 - acc: 0.9841 - val_loss: 0.0412 - val_acc: 0.9839
Epoch 6/10
127656/127656 [==============================] - 77s 603us/step - loss: 0.0393 - acc: 0.9845 - val_loss: 0.0410 - val_acc: 0.9840
Epoch 7/10
127656/127656 [==============================] - 77s 600us/step - loss: 0.0379 - acc: 0.9850

Epoch 8/10
127657/127657 [==============================] - 76s 594us/step - loss: 0.0363 - acc: 0.9853 - val_loss: 0.0392 - val_acc: 0.9848
Epoch 9/10
127657/127657 [==============================] - 76s 597us/step - loss: 0.0352 - acc: 0.9858 - val_loss: 0.0394 - val_acc: 0.9850
Epoch 10/10
127657/127657 [==============================] - 76s 597us/step - loss: 0.0344 - acc: 0.9860 - val_loss: 0.0404 - val_acc: 0.9846
Train on 127657 samples, validate on 31914 samples
Epoch 1/10
127657/127657 [==============================] - 78s 612us/step - loss: 0.0618 - acc: 0.9787 - val_loss: 0.0451 - val_acc: 0.9826
Epoch 2/10
127657/127657 [==============================] - 77s 604us/step - loss: 0.0460 - acc: 0.9825 - val_loss: 0.0437 - val_acc: 0.9834
Epoch 3/10
127657/127657 [==============================] - 77s 604us/step - loss: 0.0430 - acc: 0.9833 - val_loss: 0.0419 - val_acc: 0.9836
Epoch 4/10
127657/127657 [==============================] - 77s 607us/step - loss: 0.0409 - acc: 0.984

In [5]:
sample_submission[list_classes] = test_pred
sample_submission.to_csv("../results/lstm_attention_glove_sample_5.gz", index=False, compression='gzip')
with open('../features/lstm_attention_glove_5_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)
print(sample_submission.head())
print('===================================')

                 id     toxic  severe_toxic   obscene        threat    insult  \
0  00001cee341fdb12  0.998212  3.448951e-01  0.954187  6.623778e-02  0.899314   
1  0000247867823ef7  0.000408  2.949290e-06  0.000066  2.324186e-06  0.000036   
2  00013b17ad220c46  0.000704  2.330996e-05  0.000082  7.191057e-06  0.000061   
3  00017563c3f7919a  0.000022  1.528271e-07  0.000009  2.078268e-07  0.000003   
4  00017695ad8997eb  0.001567  1.808700e-05  0.000120  2.457606e-05  0.000081   

   identity_hate  
0   6.095416e-01  
1   9.335724e-06  
2   2.464245e-05  
3   3.283043e-07  
4   2.472824e-05  
